# Convert SQLite output to parquet file with CytoTable

## Import libraries

In [1]:
import pathlib
import pandas as pd

# cytotable will merge objects from SQLite file into single cells and save as parquet file
from cytotable import convert, presets

import logging

# Set the logging level to a higher level to avoid outputting unnecessary errors from config file in convert function
logging.getLogger().setLevel(logging.ERROR)

## Set paths and variables

In [2]:
# preset configurations based on typical CellProfiler outputs
preset = "cellprofiler_sqlite_pycytominer"

# update preset to include site metadata and cell counts
joins = presets.config["cellprofiler_sqlite_pycytominer"]["CONFIG_JOINS"].replace(
    "Image_Metadata_Well,",
    "Image_Metadata_Well, Image_Metadata_Site, Image_Count_Cells,",
)

# Add the PathName columns separately
joins = joins.replace(
    "COLUMNS('Image_FileName_.*'),",
    "COLUMNS('Image_FileName_.*'),\n COLUMNS('Image_PathName_.*'),",
)

# type of file output
dest_datatype = "parquet"

# set path to directory with SQLite files
sqlite_dir = pathlib.Path("../2.extract_features/cp_output")

# directory for processed data
output_dir = pathlib.Path("data")
output_dir.mkdir(parents=True, exist_ok=True)

plate_names = []

for file_path in sqlite_dir.iterdir():
    plate_names.append(file_path.stem)

# print the plate names and how many plates there are (confirmation)
print(f"There are {len(plate_names)} plates in this dataset. Below are the names:")
for name in plate_names:
    print(name)

There are 1 plates in this dataset. Below are the names:
CARD-CelIns-CX7_251023130003


## Convert SQLite to parquet files

In [3]:
for file_path in sqlite_dir.iterdir():
    output_path = pathlib.Path(
        f"{output_dir}/converted_profiles/{file_path.stem}_converted.parquet"
    )
    print("Starting conversion with cytotable for plate:", file_path.stem)
    # Merge single cells and output as parquet file
    convert(
        source_path=str(file_path),
        dest_path=str(output_path),
        dest_datatype=dest_datatype,
        preset=preset,
        joins=joins,
        chunk_size=5000,
    )

print("All plates have been converted with cytotable!")

Starting conversion with cytotable for plate: CARD-CelIns-CX7_251023130003
All plates have been converted with cytotable!


# Load in converted profiles to update

In [4]:
# Directory with converted profiles
converted_dir = pathlib.Path(f"{output_dir}/converted_profiles")

for file_path in converted_dir.iterdir():
    # Load the DataFrame from the Parquet file
    df = pd.read_parquet(file_path)

    # If any, drop rows where "Metadata_ImageNumber" is NaN (artifact of cytotable)
    df = df.dropna(subset=["Metadata_ImageNumber"])

    # Rearrange columns and add "Metadata" prefix in one line
    df = df[
        [
            "Nuclei_Location_Center_X",
            "Nuclei_Location_Center_Y",
            "Cells_Location_Center_X",
            "Cells_Location_Center_Y",
            "Image_Count_Cells",
        ]
        + [
            col
            for col in df.columns
            if col
            not in [
                "Nuclei_Location_Center_X",
                "Nuclei_Location_Center_Y",
                "Cells_Location_Center_X",
                "Cells_Location_Center_Y",
                "Image_Count_Cells",
            ]
        ]
    ].rename(
        columns=lambda col: (
            "Metadata_" + col
            if col
            in [
                "Nuclei_Location_Center_X",
                "Nuclei_Location_Center_Y",
                "Cells_Location_Center_X",
                "Cells_Location_Center_Y",
                "Image_Count_Cells",
            ]
            else col
        )
    )

    # Save the processed DataFrame as Parquet in the same path
    df.to_parquet(file_path, index=False)

## Check output to confirm process worked

To confirm the number of single cells is correct, please use any database browser software to see if the number of rows in the "Per_Cells" compartment matches the number of rows in the data frame.

In [5]:
converted_df = pd.read_parquet(
    "./data/converted_profiles/CARD-CelIns-CX7_251023130003_converted.parquet"
)

print(converted_df.shape)
converted_df.head()

(12158, 2476)


,Metadata_Nuclei_Location_Center_X,Metadata_Nuclei_Location_Center_Y,Metadata_Cells_Location_Center_X,Metadata_Cells_Location_Center_Y,Metadata_Image_Count_Cells,Metadata_ImageNumber,Image_Metadata_Plate,Image_Metadata_Site,Image_Metadata_Well,Metadata_Cells_Number_Object_Number,...,Nuclei_Texture_Variance_Hoechst_3_02_256,Nuclei_Texture_Variance_Hoechst_3_03_256,Nuclei_Texture_Variance_Mitochondria_3_00_256,Nuclei_Texture_Variance_Mitochondria_3_01_256,Nuclei_Texture_Variance_Mitochondria_3_02_256,Nuclei_Texture_Variance_Mitochondria_3_03_256,Nuclei_Texture_Variance_PM_3_00_256,Nuclei_Texture_Variance_PM_3_01_256,Nuclei_Texture_Variance_PM_3_02_256,Nuclei_Texture_Variance_PM_3_03_256
0,222.515520,91.821668,227.192782,133.566315,13,8,CARD-CelIns-CX7_251023130003,f09,B02,1,...,6.456837,6.374919,0.507924,0.503667,0.532190,0.508049,1.454409,1.388667,1.442516,1.428607
1,364.706757,76.552703,364.512899,114.433453,8,18,CARD-CelIns-CX7_251023130003,f19,B02,1,...,126.824810,128.172289,253.705013,255.674477,241.869275,238.018235,192.838915,193.238477,179.877037,181.075333
2,666.890013,130.611252,672.886981,123.623609,6,27,CARD-CelIns-CX7_251023130003,f06,B03,1,...,6.010343,5.910654,1.187196,1.189512,1.345693,1.092350,1.067978,1.066763,1.049050,1.053286
3,123.713090,154.224505,134.946109,181.746611,8,36,CARD-CelIns-CX7_251023130003,f18,B03,1,...,26.468006,25.320270,2.922932,2.788681,2.784761,2.638553,1.242155,1.160329,1.294659,1.258544
4,80.011967,200.934181,144.184192,207.999922,11,37,CARD-CelIns-CX7_251023130003,f19,B03,1,...,21.551625,20.193449,2.876889,2.807296,3.005361,2.920501,4.535811,4.277873,4.300654,4.117515


In [13]:
col = "Metadata_ImageNumber"
mask = converted_df[col] == 52

print(f"ImageNumber 52 found: {bool(mask.any())}")
print(f"Number of rows with ImageNumber 52: {int(mask.sum())}")

if mask.any():
    print(
        "Wells for ImageNumber 52:",
        converted_df.loc[mask, "Image_Metadata_Well"].unique(),
    )
    display(converted_df.loc[mask].head())

ImageNumber 52 found: True
Number of rows with ImageNumber 52: 8
Wells for ImageNumber 52: ['B04']


,Metadata_Nuclei_Location_Center_X,Metadata_Nuclei_Location_Center_Y,Metadata_Cells_Location_Center_X,Metadata_Cells_Location_Center_Y,Metadata_Image_Count_Cells,Metadata_ImageNumber,Image_Metadata_Plate,Image_Metadata_Site,Image_Metadata_Well,Metadata_Cells_Number_Object_Number,...,Nuclei_Texture_Variance_Hoechst_3_02_256,Nuclei_Texture_Variance_Hoechst_3_03_256,Nuclei_Texture_Variance_Mitochondria_3_00_256,Nuclei_Texture_Variance_Mitochondria_3_01_256,Nuclei_Texture_Variance_Mitochondria_3_02_256,Nuclei_Texture_Variance_Mitochondria_3_03_256,Nuclei_Texture_Variance_PM_3_00_256,Nuclei_Texture_Variance_PM_3_01_256,Nuclei_Texture_Variance_PM_3_02_256,Nuclei_Texture_Variance_PM_3_03_256
584,359.440320,160.431587,352.284008,184.273628,8,52,CARD-CelIns-CX7_251023130003,f14,B04,1,...,7.788926,7.883822,0.955434,0.900322,0.949005,0.934394,1.958895,1.833154,1.767795,1.797568
1679,521.347633,310.166420,524.976237,290.490048,8,52,CARD-CelIns-CX7_251023130003,f14,B04,2,...,10.174384,10.037300,0.880555,0.718635,0.712417,0.735800,12.055240,12.317824,11.956822,12.671070
2320,734.030354,382.338954,724.957451,377.914307,8,52,CARD-CelIns-CX7_251023130003,f14,B04,3,...,8.857952,8.772660,0.423683,0.414435,0.421338,0.387376,1.496618,1.492611,1.577405,1.461463
3546,615.225265,473.916961,610.598699,477.372824,8,52,CARD-CelIns-CX7_251023130003,f14,B04,4,...,11.550005,11.628175,1.285020,1.291212,1.374733,1.302622,1.224379,1.188453,1.162337,1.148225
4432,523.464438,479.940967,533.962442,528.188620,8,52,CARD-CelIns-CX7_251023130003,f14,B04,5,...,6.487833,6.508727,1.274485,1.293183,1.234112,1.173280,1.087795,1.098441,1.140950,1.097714
